# Data Cleaning: NYC 311 Service Requests

This notebook downloads and cleans the 2024 NYC 311 service request data. We start by fetching the raw data from the Socrata API, then apply cleaning steps to prepare it for analysis.

## Installing required packages

The next cell installs the `python-dotenv` package, which allows us to load environment variables from the `.env` file. This keeps our API token secure and out of the code. It also loads some other required packages for the notebook to run

In [ ]:
#!python3 -m pip install python-dotenv
#!python3 -m pip install pandas 
#!python3 -m pip install seaborn matplotlib plotly scikit-learn pyarrow

## Fetching and saving raw data

The next cell loads the environment variables, imports the data fetching function, and retrieves all 2024 NYC 311 records from the Socrata API. It handles pagination automatically and prints summary information about the fetched data. Finally, it saves the raw dataset as a parquet file for later cleaning steps.

In [ ]:
# importing fetch function for NYC 311
from pathlib import Path
import os
import sys

# loading environment variables
try:
    from dotenv import load_dotenv
    load_dotenv(Path('..') / '.env')
except ImportError:
    print("Warning: python-dotenv not installed. Install with: pip install python-dotenv")
    print("Continuing without .env file support...")

PROJECT_ROOT = Path('..').resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

from src.nyc311_api import fetch_nyc311_data

# reading Socrata app token from environment variable
APP_TOKEN = os.getenv("SOCRATA_APP_TOKEN")
if not APP_TOKEN:
    print("Note: SOCRATA_APP_TOKEN not found in environment. API calls may be slower.")

# fetching 2024 data
df_raw = fetch_nyc311_data(
    app_token=APP_TOKEN,
    limit=25000,  
    year=2024,
)

print(f"DataFrame shape: {df_raw.shape}")
print(f"Columns: {list(df_raw.columns)}")
print(f"\nFirst few rows:")
print(df_raw.head().to_string())

raw_dir = PROJECT_ROOT / 'data' / 'raw'
raw_dir.mkdir(parents=True, exist_ok=True)

raw_path = raw_dir / 'nyc311_2024_raw.parquet'
df_raw.to_parquet(raw_path, index=False)

print(f"\nRaw data saved to: {raw_path}")
raw_path

Starting data fetch for year 2024...
Using limit of 25,000 rows per request.
App token detected. Using authenticated requests.

Fetching page 1 (offset 0)... Received 25,000 rows (total so far: 25,000)
Fetching page 2 (offset 25,000)... Received 25,000 rows (total so far: 50,000)
Fetching page 3 (offset 50,000)... Received 25,000 rows (total so far: 75,000)
Fetching page 4 (offset 75,000)... Received 25,000 rows (total so far: 100,000)
Fetching page 5 (offset 100,000)... Received 25,000 rows (total so far: 125,000)
Fetching page 6 (offset 125,000)... Received 25,000 rows (total so far: 150,000)
Fetching page 7 (offset 150,000)... Received 25,000 rows (total so far: 175,000)
Fetching page 8 (offset 175,000)... Received 25,000 rows (total so far: 200,000)
Fetching page 9 (offset 200,000)... Received 25,000 rows (total so far: 225,000)
Fetching page 10 (offset 225,000)... Received 25,000 rows (total so far: 250,000)
Fetching page 11 (offset 250,000)... Received 25,000 rows (total so far: 

PosixPath('/Users/sagorika/Documents/HCDS/Final Project/A7/data/raw/nyc311_2024_raw.parquet')

The next cell just outputs first few rows of the dataframe

In [ ]:
print(df_raw.head().to_string())

  unique_key             created_date              closed_date agency                              agency_name     complaint_type descriptor               location_type incident_zip            incident_address             street_name     cross_street_1  cross_street_2 intersection_street_1 intersection_street_2 address_type           city                landmark  status                                                                                                                                                                              resolution_description resolution_action_updated_date   community_board         bbl        borough x_coordinate_state_plane y_coordinate_state_plane open_data_channel_type park_facility_name   park_borough            latitude           longitude                                                                                                                                       location facility_type bridge_highway_name bridge_highway_segment taxi_pic

## Loading raw data from disk

The next cell sets up the project paths and loads the raw data from the saved parquet file. We need to use this cell instead of the API fetch cell if we've already downloaded the data.

In [ ]:
from pathlib import Path
import pandas as pd
import sys

PROJECT_ROOT = Path('..').resolve()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

# loading raw data from parquet file
raw_path = PROJECT_ROOT / 'data' / 'raw' / 'nyc311_2024_raw.parquet'
if raw_path.exists():
    print(f"Loading raw data from {raw_path}...")
    df_raw = pd.read_parquet(raw_path)
    print(f"Loaded {len(df_raw):,} rows")
    print(f"DataFrame shape: {df_raw.shape}")
else:
    print(f"Raw data file not found at {raw_path}")
    print("Please run the data fetching cell (Cell 4) first to download the data.")

Loading raw data from /Users/sagorika/Documents/HCDS/Final Project/A7/data/raw/nyc311_2024_raw.parquet...
Loaded 3,458,320 rows
DataFrame shape: (3458320, 41)


## Examining data types and structure

The next cell checks the data types of each column and shows basic information about missing values. This helps us understand what cleaning steps are needed.


In [ ]:
print("Data types:")
print(df_raw.dtypes)
print("\n" + "="*80)
print("\nMissing values per column:")
missing = df_raw.isnull().sum()
missing_pct = (missing / len(df_raw) * 100).round(2)
missing_df = pd.DataFrame({
    'Missing Count': missing,
    'Missing %': missing_pct
})
print(missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False))
print("\n" + "="*80)
print(f"\nTotal rows: {len(df_raw):,}")
print(f"Key columns for analysis:")
print(f"  - created_date: {df_raw['created_date'].isnull().sum():,} missing")
print(f"  - closed_date: {df_raw['closed_date'].isnull().sum():,} missing")
print(f"  - borough: {df_raw['borough'].isnull().sum():,} missing")
print(f"  - open_data_channel_type: {df_raw['open_data_channel_type'].isnull().sum():,} missing")
print(f"  - complaint_type: {df_raw['complaint_type'].isnull().sum():,} missing")

Data types:
unique_key                        object
created_date                      object
closed_date                       object
agency                            object
agency_name                       object
complaint_type                    object
descriptor                        object
location_type                     object
incident_zip                      object
incident_address                  object
street_name                       object
cross_street_1                    object
cross_street_2                    object
intersection_street_1             object
intersection_street_2             object
address_type                      object
city                              object
landmark                          object
status                            object
resolution_description            object
resolution_action_updated_date    object
community_board                   object
bbl                               object
borough                           object
x_co

## Cleaning the data

The next cell imports the cleaning functions and applies all cleaning steps to prepare the data for analysis. This includes converting dates, computing response times, filtering invalid rows, standardizing channel types, removing duplicates, winsorizing outliers, and selecting top complaint types.

In [4]:
import sys, pandas as pd
from pathlib import Path

PROJECT_ROOT = Path('..').resolve()
sys.path.insert(0, str(PROJECT_ROOT))
from src.nyc311_cleaning import clean_nyc311_data

# keeping top 10 complaint types by default for reliable comparisons
df_clean = clean_nyc311_data(
    df_raw,
    winsorize_percentile=99.0,
    top_complaint_types=10,
)

print("Cleaned data summary:")
print(f"Shape: {df_clean.shape}")
print(f"\nResponse time statistics (days):")
print(df_clean['response_time_days'].describe())
print(f"\nChannel distribution:")
print(df_clean['open_data_channel_type'].value_counts())
print(f"\nBorough distribution:")
print(df_clean['borough'].value_counts())

Starting data cleaning process...
Initial row count: 3,458,320

Step 1: Converting dates to datetime...
Step 2: Computing response time...
Step 3: Filtering invalid rows...
Removed 128,060 rows with missing required fields or invalid response times.
Remaining rows: 3,330,260
Step 4: Standardizing channel types...

Channel type distribution after standardization:
open_data_channel_type
Web      1586605
Phone    1032985
App       710670
Name: count, dtype: int64
Step 5: Removing duplicates...

Removed 2 duplicate rows based on unique_key.
Remaining rows: 3,330,258
Step 6: Winsorizing response times at 99.0th percentile...

Winsorized 33,303 values at 99.0th percentile (273.21 days).
Step 7: Selecting top complaint types...

Filtered to top complaint types:
  Kept 10 complaint types
  Removed 1,460,942 rows from other complaint types
  Remaining rows: 1,869,316

Top complaint types:
  - Illegal Parking: 505,732 records
  - Noise - Residential: 379,252 records
  - HEAT/HOT WATER: 261,748 r

## Saving cleaned data

The next cell saves the cleaned dataset to the processed data folder for use in the main analysis notebook.

In [ ]:
processed_dir = PROJECT_ROOT / 'data' / 'processed'
processed_dir.mkdir(parents=True, exist_ok=True)

clean_path = processed_dir / 'nyc311_2024_cleaned.parquet'
df_clean.to_parquet(clean_path, index=False)

print(f"Cleaned data saved to: {clean_path}")
print(f"Final dataset: {len(df_clean):,} rows, {len(df_clean.columns)} columns")
clean_path

Cleaned data saved to: /Users/sagorika/Documents/HCDS/Final Project/A7/data/processed/nyc311_2024_cleaned.parquet
Final dataset: 1,869,316 rows, 42 columns


PosixPath('/Users/sagorika/Documents/HCDS/Final Project/A7/data/processed/nyc311_2024_cleaned.parquet')